To dowload the jupyther file to python script use:

In [1]:
# Descargar script
#!jupyter nbconvert --to python actualizar_datos.ipynb

In [4]:
# Import libraries
from __future__ import print_function
import os.path
import pandas as pd
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from  _general import clean_text
import spacy
import subprocess
import sys
from gensim.models.word2vec import Word2Vec
import multiprocessing
import numpy as np
from tqdm import tqdm
from annoy import AnnoyIndex
import json
from cryptography.fernet import Fernet
from datetime import date

In [3]:
# CONECTARSE A BASE DE DATOS 
# Could be cached
def connect_sheet(credentials):
    # If modifying these scopes, delete the file token.json.
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets',
              'https://www.googleapis.com/auth/drive']

    # Connect to database
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                credentials, SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    return creds


In [4]:
credential = connect_sheet("./secret/credentials.json")
credential

In [5]:
# CONSEGUIR DATOS DE BASE DE DATOS
def get_data(creds):
    """
    Connect to database using credentials. 
    Get all information about planilla (productos requeridos) and bd_cotizaciones (productos cotizados)
    """
    # The ID and range of a sample spreadsheet.
    SAMPLE_SPREADSHEET_ID = '1Ge_nrrNF9VoVMysRiGm-Is53jSMh2OL_8cxTIimfpxE'
    bd_cotizaciones = 'BD_Cotizaciones_de_proveedores!A1:Q'
    planilla = 'Planilla!A1:G'

    # Call the Sheets API
    sheet_service = build('sheets', 'v4', credentials=creds) 
    sheet = sheet_service.spreadsheets()

    # Get data from database
    bd_cotizaciones_data = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                range=bd_cotizaciones).execute()
    bd_cotizaciones_data_values = bd_cotizaciones_data.get("values",[])

    planilla_data = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                range=planilla).execute()
    planilla_data_values = planilla_data.get("values",[])

    # Save as pandas dataframe
    df_bd = pd.DataFrame.from_records(bd_cotizaciones_data_values[1:],columns=bd_cotizaciones_data_values[0])
    df_planilla = pd.DataFrame.from_records(planilla_data_values[1:],columns=planilla_data_values[0])

    # Drop empty lines
    df_bd['TRAZA'] = df_bd['TRAZA'].replace('', np.nan)
    df_planilla['TRAZA'] = df_planilla['TRAZA'].replace('', np.nan)
    df_bd = df_bd.dropna(subset=['TRAZA'])
    df_planilla = df_planilla.dropna(subset=['TRAZA'])
    
    return df_bd, df_planilla


In [6]:
df_productos, df_planilla = get_data(credential)
df_productos

,Cotizacion,Producto Solicitado,Producto Ofrecido,U. Medida,Cantidad,Costo x Unidad,Costo x Unidad SIN IGV en Soles,Precio Venta x Unidad,Link,Observaciones,Habilitar y Deshabilitar,Adjunto,Adjunto Ficha Tecnica,Imagen Referencial,Ultima revisión,TRAZA,OC a proveedor
0,47630ddf50e6f0e4,5a76b72b,"MONITOR LG 21.5’’ (22MN430M-B), 1920 X 1080 (F...",Unidad,2,550.5,466.53,601.17,,,Deshabilitado,,,,,152e43f1,None
1,47630ddf50e6f0e4,3d56f01b,TECLADO INALAMBRICO + MOUSE INALAMBRICO GENIU...,Unidad,15,78.4,66.44,82.1,,,Deshabilitado,,,,,1ac5849a,None
2,47630ddfb5351092,3d56f01b,TECLADO INALAMBRICO + MOUSE INALAMBRICO TEROS...,Unidad,15,59.9,50.76,82.1,https://dasmitec.pe/productos/kit-inalambrico-...,,Deshabilitado,,,,,d79583f9,None
3,47630ddf05708cfd,3d56f01b,COMBO LOGITECH: TECLADO + MOUSE MK220 WIRELESS...,Unidad,15,77.5,77.5,82.1,https://www.memorykings.com.pe/producto/327993...,,Deshabilitado,,,,,aedc03a3,None
4,47630ddf05708cfd,5a76b72b,MONITOR 22” SAMSUNG LF22T350FHLXPE iPS FHD HDM...,Unidad,2,567.5,567.5,601.17,,,Habilitado,,,,,398d553b,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8735,f1ab26fa,97df0ae1,PLANCHA DE EMPASTAR M/GOMAAMARILLO WINGS 30 cm,Unidad,3,10.38,8.7966,0,,,Deshabilitado,,,,,a6443415,None
8736,f1ab26fa,97df0ae1,PLANCHA DE EMPASTA M/GOMAEUROTOLLS 30 cm,Unidad,3,9.51,8.0593,0,,,Deshabilitado,,,,,5db96e92,None
8737,f1ab26fa,97df0ae1,PLANCHA DE EMPASTAR M/GOMA ROJA PROFIELD 30 CM,Unidad,3,10.7,9.0678,0,,,Deshabilitado,,,,,f70115b3,None
8738,a601735d,431ff4cb,VNMG 16 04 04-QM 4325 T-Max P P Rómbica 35º In...,Unidad,10,33.28,121.472,0,,,Deshabilitado,,,,,92494ff0,None


In [35]:
def save_data(df):
    """ Save pandas dataframe
    """

    # key generation
    key = Fernet.generate_key()
    
    # save the key in a file
    with open('./data/filekey.key', 'wb') as filekey:
        filekey.write(key)

    # encrypting the dataframe and saving 
    f = Fernet(key)
    df_e = df.apply(lambda x: x.astype(str)) # preprocess
    # Encrypt headers
    columns = list(df_e.columns.values)

    columns = map(lambda x: f.encrypt(x.encode('utf-8')),columns)
    df_e.columns = columns

    # Encrypt content
    token = df_e.applymap(lambda x: f.encrypt(x.encode('utf-8')))
    
    
    token.to_csv('./data/productos.csv', index=False)
        

    # DEVELOPING
    # Goal: to obtain latest data from database and create to new index and model
    # # Save to google sheet
    # gc = gspread.authorize(creds)
    # spreadsheet_key = '1DaarZ_7EjFEMO-5hZsRB5J7FWq1eVdKm4fdld0Ww5Lc'
    # workbook = gc.open_by_key(spreadsheet_key)
    # workbook.values_clear("A:P")
    # workbook.values_update(
    # 'productos_cotizados!A1',
    # params={
    #     'valueInputOption': 'USER_ENTERED'
    # },
    # body={
    #     'values': [df_bd.columns.values.tolist()] + df_bd.values.tolist()
    # }

In [36]:
save_data(df_productos)

In [ ]:
df_planilla

In [ ]:
# NORMALIZAR NOMBRES
def normalize(df,columna_producto):
    new_column ="Norm_" + columna_producto 
    df[new_column] = [clean_text(producto,True,True,True) for producto in df[columna_producto]]

    return df

In [ ]:
df_productos = normalize(df_productos,"Producto Ofrecido")
df_productos

In [ ]:
df_planilla = normalize(df_planilla,"Categoria")
df_planilla = normalize(df_planilla,"Producto Solicitado")
df_planilla

In [ ]:
# COMBINAR NOMBRES DFs
def combinar_bd(df1,df2,column1,column2):
    """
    Combines two pandas DFs

    ** df1 = pandas DF al que añadir valores
    ** df2 = pandas DF del que se sacaran valores
    ** column1,2 = Str. Nombre de la columna con el idx de connecion

    Lista a conseguir
    # Producto solicitado
    # Producto ofrecido
    # Categorias
    # Producto solicitado con categorias
    # Producto ofrecido con categorias
    # Producto solicitado con categoria y todos los prod ofrecidos
    """
    # Combinar dfs
    df = pd.merge(df1, df2,
                       how='left', left_on=column1, right_on=column2)
    
    # Make TRAIN_DATA
    TRAIN_DATA = []
    # Producto solicitado
    TRAIN_DATA = TRAIN_DATA + df["Norm_Producto Solicitado"].values.tolist() 
    
    # Producto ofrecido
    TRAIN_DATA = TRAIN_DATA + df["Norm_Producto Ofrecido"].values.tolist() 
    
    # Categorias
    TRAIN_DATA = TRAIN_DATA + df["Norm_Categoria"].values.tolist() 
    
    # Producto solicitado con categorias
    df["sol_categoria"] = df["Norm_Producto Solicitado"]+ " "+ df["Norm_Categoria"]
    TRAIN_DATA = TRAIN_DATA + df["sol_categoria"].values.tolist() 
    
    # Producto ofrecido con categorias
    df["ofr_categoria"] = df["Norm_Producto Ofrecido"]+ " "+ df["Norm_Categoria"]
    TRAIN_DATA = TRAIN_DATA + df["ofr_categoria"].values.tolist() 
    
    # Producto solicitado con categoria y todos los prod ofrecidos
    df["todo"] = df["Norm_Producto Solicitado"]+ " " + df["Norm_Categoria"] + " " +df["Norm_Producto Ofrecido"]
    TRAIN_DATA = TRAIN_DATA + df["todo"].values.tolist()  
    
    # Remove duplicates
    TRAIN_DATA_NEW = [] 
    for i in TRAIN_DATA:
        if isinstance(i,str):
            i = i.strip()
            if i not in TRAIN_DATA_NEW: 
                TRAIN_DATA_NEW.append(i) 

    
    return TRAIN_DATA_NEW


In [ ]:
TRAIN_DATA = combinar_bd(df_productos,df_planilla,"Producto Solicitado","TRAZA")
TRAIN_DATA

In [ ]:
# CREAR VECTORES
def create_vectors(data, creds):
    """
    Crea gensim vectors
    ** data: list of words normalized and without stop words
    """

    # Split data
    texts = []
    for producto in data:
        if isinstance(producto,str) and len(producto)>0: 
            producto_split = producto.split(" ")
            palabras = []
            for word in producto_split:
                word = word.strip()
                
                if len(word) > 0:
                    palabras.append(word)
            texts.append(palabras)

    # Create wordVector
    cores = multiprocessing.cpu_count()
    w2v_model = Word2Vec(min_count=3,
                            window=2,
                            vector_size=500,
                            sample=6e-5,
                            alpha=0.03,
                            min_alpha=0.0007,
                            negative=20,
                            workers=cores-1)
    w2v_model.build_vocab(texts)
    w2v_model.train(texts, total_examples=w2v_model.corpus_count,epochs=30)

    # Save data
    # THE GOAL WAS TO SAVE THE NEW MODEL TO THE CLOUD.
    #w2v_model.wv.save_word2vec_format("data/vectores.txt")
        
    # # create drive api client
    # service = build('drive', 'v3', credentials=creds)

    # body = {'name': 'texto.txt', 'parents':["1pB11fza6UjGBguDcyIa9vhXcHnyPJFR6"]}
    # media = MediaFileUpload('./data/vectores.txt',
    #                         mimetype='text/plain')
    
    # #Now we're doing the actual post, creating a new file of the uploaded type
    # file = service.files().create(body=body, media_body=media).execute()

    # #Because verbosity is nice
    # print ("Created file '%s' id '%s'." % (file.get('name'), file.get('id')))
    


In [ ]:
create_vectors(TRAIN_DATA,credential)

In [ ]:
# COPIAR VECTORS EN MODELO DE SPACY
def load_word_vectors():

    model_name = "./model"

    nlp = spacy.blank("es-419")
    nlp.to_disk(model_name)


    subprocess.run([sys.executable,"-m","spacy",
                    "init", "vectors","es","data/vectores.txt",model_name])
    

In [ ]:
load_word_vectors()

In [ ]:
# USAR LISTA DE PRODUCTOS OFRECIDOS PARA CREAR INDICE
def create_index(df_productos):
    """ 
    Takes: pandas Data Frame
    Returns: Nan
    Void: Saves annoy index with vectors of each product
    """

    # Load model
    nlp = spacy.load("./model")

    # Get data from df
    train_list = df_productos[["Norm_Producto Ofrecido","TRAZA"]].values.tolist() 
    train_list = dict(zip(df_productos["TRAZA"], df_productos["Norm_Producto Ofrecido"]))

    # Create annoy object
    annoy_index = AnnoyIndex(500,'euclidean')

    # Make a dictionary for reference data
    reference = {}

    # For every produtct
    for ix,(traza,text) in tqdm(enumerate(train_list.items())):
        if text != None:

            # Convert to vector
            doc = nlp(text) 

            # Add to annoy index
            annoy_index.add_item(ix, doc.vector)

            # Add data to reference dictionary
            reference[ix] = (traza,text)
    annoy_index.build(10)

    # THE GOAL WAS TO UPDATE THE VALUES OF THE NEW INDEX WITH THE NEW DATA TO THE CLOUD
    # Save data
    # NEXT STEPS:
    # 1. Create temporary directory
    # https://stackoverflow.com/questions/3223604/how-do-i-create-a-temporary-directory-in-python
    # 2. Use it to run a subprocess to google drive
    # https://discuss.streamlit.io/t/accessing-temporary-files-on-streamlit-sharing/16221/4
    # https://stackoverflow.com/questions/65071321/save-and-load-a-spacy-model-to-a-google-cloud-storage-bucket

    
    annoy_index.save('./data/product_index.ann')

    with open("./data/reference_map.json", "w",encoding="utf-8") as f:
        json.dump(reference, f, indent = 4)
   
    return 

In [ ]:
create_index(df_productos)

In [8]:
# TIME STAMP
with open("./data/time_stamp.json", "w",encoding="utf-8") as f:
    today = str(date.today())
    json.dump(today, f, indent = 4)

2023-07-17
<class 'str'>
